In [1]:
# !pip install webdriver_manager

In [1]:
#크롤링시 필요한 라이브러리 불러오기
import pandas as pd
from bs4 import BeautifulSoup #HTML tag를 쉽게 분석해주는 패키지
import request #API request를 호출할 수 있도록 도와주는 패키지
import re #정규표현식으로 전처리를 돕는 패키지
import time 
from selenium import webdriver #웹브라우저 컨트롤
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm #코드 진행률 확인


#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [2]:
query_data = pd.read_excel("../data/이태원 1027_1118.xlsx")

빅카인즈에서 수집한 데이터 \
여기서 뉴스 제목을 네이버에 검색하도록 하면 해당 뉴스의 naver뉴스 url를 얻을 확률이 높음

In [4]:
query_data.head(2)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,1.300101e+06,20221118,강원도민일보,남궁창성,"국민의힘 권성동 ""MBC 자막조작은 청담동 술자리 괴담과 동급""",사회>미디어,정치>청와대,정치>국회_정당,NaN,NaN,NaN,NaN,"청담동,이태원,용산","한미동맹,가짜,대국,페이스북,MBC,국민의힘,도어스","국민의힘,권성동,MBC,자막,조작,괴담,청담동,술자리,동급,악의,보도,논평,재난,정...","mbc,악의적,권성동,대통령실,윤석열,문제점,청담동,가짜뉴스,술자리,정쟁화,pd수첩...","'악의적 보도' 논평 ""재난 정쟁화 PD수첩은 '민들레' 행태와 본질적으로 같아""용...",www.kado.net/news/articleView.html?idxno=1155233,NaN
1,1.100611e+06,20221118,서울신문,강민혜,"‘이태원 참사’ 유족들, CCTV 증거 보전 신청 행안부 경찰청 등",미분류,NaN,NaN,NaN,NaN,NaN,이태원,"서울,이태원","서울소방재난본부,경찰,서울경찰청,이태원파출소,법원,태스크,서울종합방재센터,행정안전부...","이태원,참사,유족들,CCTV,증거,보전,신청,행안부,경찰청,민주사회,변호사모임,민변...","민변,보고서,희생자,태스크포스,경찰청,서울,cctv,멸실,신청서,서울종합방재센터,웨...",민주사회를 위한 변호사모임(민변)이 ‘이태원 참사’ 희생자 유가족을 대리해 참사 현...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN


우선 어떤 변수들이 존재하는지 확인

In [5]:
query_data.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [6]:
query_lst = query_data[['제목', '뉴스 식별자']] #검색어로 사용할 제목과 나중에 원본 data와 매칭하기 위한 뉴스 식별자
print(len(query_lst))

20000


In [7]:
query_lst

,제목,뉴스 식별자
0,"국민의힘 권성동 ""MBC 자막조작은 청담동 술자리 괴담과 동급""",1.300101e+06
1,"‘이태원 참사’ 유족들, CCTV 증거 보전 신청 행안부 경찰청 등",1.100611e+06
2,[포토] ‘오늘은 정쟁 대신 볼다툼’...여야 의원 친선 축구대회,1.100611e+06
3,이태원 참사 트라우마 치유 ‘사각지대’ 놓인 외국인 유학생들,1.100101e+06
4,"민주당 '참사 진상규명' 압박 국민의힘 ""이재명 퇴진 운동이 먼저""",1.101101e+06
...,...,...
19995,"[이태원 참사] 경찰, 허위사실 유포 11건 조사 인파사고TF 설치",2.100701e+06
19996,"삼성전자, 조용한 창립기념일 추모 묵념으로 기념식 시작",1.100401e+06
19997,이태원 참사 합동분향소 찾은 국민의힘,2.100501e+06
19998,"""주최자없는 행사, 지자체가 안전관리"" 서울시의회 조례 만든다",2.100201e+06


In [8]:
##########뉴스크롤링 시작###################

search_urls = [] #찾은 url list를 저장할 빈 리스트 생성

#검색어 입력
for query in query_lst['제목']: #쿼리 리스트에 있는 뉴스 제목을 하나씩 꺼내서
    # 검색식이 들어간 naver url을 생성
    # 실제로 네이버에 특정 키워드를 검색하면 url이 아래와 같이 바뀐다는 패턴을 경험을 통해 발견해야함
    # 운이 좋으면 패턴을 발견하는거고 운나쁘면 패턴이 없을수도 있지만
    # 웹스크래핑은 결국 짬과 운이 모두 따라야하는 것이기 때문에
    # 우여곡절이 많을수록 패턴을 더 빠르게 발견할 수 있음
    search_urls.append("https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&start=" + str(1))
    
print(len(search_urls))

20000


In [9]:
# 뉴스 제목 대신 뉴스 제목으로 검색한 naver url을 사용할 예정
query_lst['제목'] = search_urls

/tmp/ipykernel_2973/1775808015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_lst['제목'] = search_urls


In [11]:
query_lst

,제목,뉴스 식별자
0,https://search.naver.com/search.naver?where=ne...,1.300101e+06
1,https://search.naver.com/search.naver?where=ne...,1.100611e+06
2,https://search.naver.com/search.naver?where=ne...,1.100611e+06
3,https://search.naver.com/search.naver?where=ne...,1.100101e+06
4,https://search.naver.com/search.naver?where=ne...,1.101101e+06
...,...,...
19995,https://search.naver.com/search.naver?where=ne...,2.100701e+06
19996,https://search.naver.com/search.naver?where=ne...,1.100401e+06
19997,https://search.naver.com/search.naver?where=ne...,2.100501e+06
19998,https://search.naver.com/search.naver?where=ne...,2.100201e+06


In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # 크롬 드라이버를 사용하여 스크래핑 하겠습니다
driver.implicitly_wait(3) # 크롬 드라이버는 특정 활동 이후 3초정도 term을 두고 다음 행동을 시작합니다.


# selenium으로 검색 페이지 불러오기 #
naver_urls=[] # 수집한 뉴스 url을 저장할 빈 리스트 생성
vids = [] # 수집 성공한 뉴스의 id를 저장할 빈 리스트 생성

for title, vid in tqdm(zip(query_lst['제목'], query_lst['식별자'])): # 뉴스 검색식과 뉴스 식별자 아이디를 묶어서 하나씩 전달
    driver.switch_to.window(driver.window_handles[0]) #첫번째 탭으로 돌아오기
    driver.get(title) # 주어진 뉴스 검색식 url로 검색
    driver.implicitly_wait(0.5) # 랙걸릴것을 대비하여 잠시 대기

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR,'a.info') # 지금 페이지의 html 스크래핑 이후 필요한 정보만 저장
    
    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    if len(a) >= 2: # 뉴스가 1개 이상 있을 경우
        a[1].click() # 네이버뉴스의 형태로 제공되는 버튼을 클릭
        driver.implicitly_wait(0.5)
        driver.switch_to.window(driver.window_handles[-1]) # 방금 클릭한 탭으로 화면 전환
        driver.implicitly_wait(0.5)
        url = driver.current_url # 전환된 탭의 url을 긁어오기
        naver_urls.append(url) # 긁어온 url을 차곡차곡 저장
        vids.append(vid) # 저장에 성공했다면 뉴스 식별자도 저장
        driver.close() # 저장한 페이지 close        

print(len(naver_urls))